In [173]:
#import ANN_Weather_Forecasting

In [174]:
import pandas as pd  
import numpy as np  
import tensorflow as tf  
from sklearn.metrics import explained_variance_score,mean_absolute_error, median_absolute_error

In [175]:
#import file names
from file_names import file_cleaned, file_test,file_train



#import data

df_weather_train_set = pd.read_csv(file_train)
df_weather_train_set.describe().T
df_weather_train_set.info()
df_weather_train_set_filtered = df_weather_train_set[(df_weather_train_set['STATION_CODES'] == 2)]
df_weather_train_set_filtered=df_weather_train_set_filtered.drop('Unnamed: 0', axis=1)

# SPLIT TESTING AND VALIDATION SET BY SPLITTING LOCATIONS FOR YEARS AFTER 2005
df_weather_test_set = pd.read_csv(file_test)

# ToDo: Not sure whereS the extra 0 is coming from but must remove it
df_weather_test_set=df_weather_test_set.drop('Unnamed: 0', axis=1)
df_weather_test_set_validation = df_weather_test_set[(df_weather_test_set['STATION_CODES'] == 9)]
df_weather_test_set = df_weather_test_set[(df_weather_test_set['STATION_CODES'] == 16)]

frames = [df_weather_train_set_filtered, df_weather_test_set_validation, df_weather_test_set]
df_weather_all = pd.concat(frames)

print(df_weather_train_set_filtered.shape)
print(df_weather_test_set.shape)
print(df_weather_test_set_validation.shape)
print(df_weather_all.shape)

#combined weather dataset
X = df_weather_all[[col for col in df_weather_all.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y  = df_weather_all['AVG_DAILY_TEMP_ALL_HOURS__F']

# Train, Validate & Test sets

# X will be a pandas dataframe of all columns except AVG_DAILY_TEMP_ALL_HOURS__F
# y will be a pandas series of the AVG_DAILY_TEMP_ALL_HOURS__F
X_val = df_weather_test_set_validation[[col for col in df_weather_test_set_validation.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y_val = df_weather_test_set_validation['AVG_DAILY_TEMP_ALL_HOURS__F']

X_test = df_weather_test_set[[col for col in df_weather_test_set.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y_test = df_weather_test_set['AVG_DAILY_TEMP_ALL_HOURS__F']

X_train= df_weather_train_set_filtered[[col for col in df_weather_train_set_filtered.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y_train = df_weather_train_set_filtered['AVG_DAILY_TEMP_ALL_HOURS__F']

feature_cols = [tf.feature_column.numeric_column(col) for col in X.columns]
print(feature_cols)

X_test = X_test.drop(['DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_2','DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_3',
       'DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_1',
       'DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_2',
       'DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_3',
       'NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_2',
       'NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_3', 'COLDEST_WINDCHILL_TEMP__F_2',
       'COLDEST_WINDCHILL_TEMP__F_3', 'AVG_DAILY_TEMP_ALL_HOURS__F_1',
       'AVG_DAILY_TEMP_ALL_HOURS__F_2', 'AVG_DAILY_TEMP_ALL_HOURS__F_3',
       'NORMAL_AVG_DAILY_WET_BULB_TEMP_ALL_HOURS__F_2',
       'NORMAL_AVG_DAILY_WET_BULB_TEMP_ALL_HOURS__F_3',
       'DEPART_NORMAL__CDDS_ALL_HOURS__BASE_OF_65_F_2',
       'DEPART_NORMAL__CDDS_ALL_HOURS__BASE_OF_65_F_3', 
       'HIGH_TEMP_F_2', 'HIGH_TEMP_F_3',
       'DEPART_NORMAL__EDDS_ALL_HOURS__BASE_OF_65_F_2',
       'DEPART_NORMAL__EDDS_ALL_HOURS__BASE_OF_65_F_3',
        'WARMEST_WINDCHILL_TEMP__F_2',
       'WARMEST_WINDCHILL_TEMP__F_3'] , axis=1)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46026 entries, 0 to 46025
Data columns (total 33 columns):
Unnamed: 0                                       46026 non-null int64
AVG_DAILY_TEMP_ALL_HOURS__F                      46026 non-null float64
STATION_CODES                                    46026 non-null int64
DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_1      46025 non-null float64
DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_2      46024 non-null float64
DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_3      46023 non-null float64
DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_1      46025 non-null float64
DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_2      46024 non-null float64
DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_3      46023 non-null float64
NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_1    46025 non-null float64
NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_2    46024 non-null float64
NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_3    46023 non-null float64
COLDEST_WINDCHILL_TEMP__F_1    

In [176]:
print(X.shape)
print(y.shape)
print(X_val.shape)
print(y_val.shape)

print(X.head())

X_test.shape

(4747, 31)
(4747,)
(1095, 31)
(1095,)
      STATION_CODES  DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_1  \
5114              2                                          2.5   
5115              2                                        -10.4   
5116              2                                         -4.3   
5117              2                                         -7.7   
5118              2                                          4.0   

      DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_2  \
5114                                          3.6   
5115                                          2.5   
5116                                        -10.4   
5117                                         -4.3   
5118                                         -7.7   

      DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_3  \
5114                                          1.4   
5115                                          3.6   
5116                                          2.5   
5117                  

(1095, 9)

In [177]:
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                      hidden_units=[50,50],
                                      model_dir='tf_wx_model_v3')

def wx_input_fn(X, y=None, num_epochs=None, shuffle=True, batch_size=2400):
    return tf.estimator.inputs.pandas_input_fn(x=X,
                                               y=y,
                                               num_epochs=num_epochs,
                                               shuffle=shuffle,
                                               batch_size=batch_size)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tf_wx_model_v3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a455c6630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
evaluations = []
STEPS = 2300
for i in range(100):
    regressor.train(input_fn=wx_input_fn(X_train, y=y_train), steps=STEPS)
    evaluations.append(regressor.evaluate(input_fn=wx_input_fn(X_val,
                                                               y_val,
                                                               num_epochs=1,
                                                               shuffle=False)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into tf_wx_model_v3/model.ckpt.
INFO:tensorflow:loss = 10832926.0, step = 1
INFO:tensorflow:global_step/sec: 3.46272
INFO:tensorflow:loss = 57498.504, step = 101 (28.880 sec)
INFO:tensorflow:global_step/sec: 3.55253
INFO:tensorflow:loss = 56189.734, step = 201 (28.149 sec)
INFO:tensorflow:global_step/sec: 3.53459
INFO:tensorflow:loss = 53618.008, step = 301 (28.292 sec)
INFO:tensorflow:global_step/sec: 3.372
INFO:tensorflow:loss = 52223.875, step = 401 (29.665 sec)
INFO:tensorflow:global_step/sec: 3.38612
INFO:tensorflow:loss = 51149.28, step = 501 (29.524 sec)
INFO:tensorflow:global_step/sec: 3.36798
INFO:tensorflow:loss = 49230.086, step = 601 (29.692 sec)
INFO:tensorflow:global_step/sec: 3.39532
INFO:tensorflo

INFO:tensorflow:loss = 44743.023, step = 5601 (28.804 sec)
INFO:tensorflow:global_step/sec: 3.51898
INFO:tensorflow:loss = 43498.344, step = 5701 (28.417 sec)
INFO:tensorflow:global_step/sec: 3.51766
INFO:tensorflow:loss = 43719.25, step = 5801 (28.428 sec)
INFO:tensorflow:global_step/sec: 3.55408
INFO:tensorflow:loss = 43682.957, step = 5901 (28.137 sec)
INFO:tensorflow:global_step/sec: 3.50098
INFO:tensorflow:loss = 44158.965, step = 6001 (28.564 sec)
INFO:tensorflow:global_step/sec: 3.50619
INFO:tensorflow:loss = 43498.812, step = 6101 (28.520 sec)
INFO:tensorflow:global_step/sec: 3.54735
INFO:tensorflow:loss = 44680.89, step = 6201 (28.190 sec)
INFO:tensorflow:global_step/sec: 3.52847
INFO:tensorflow:loss = 43268.97, step = 6301 (28.340 sec)
INFO:tensorflow:global_step/sec: 3.51552
INFO:tensorflow:loss = 44530.98, step = 6401 (28.445 sec)
INFO:tensorflow:global_step/sec: 3.64871
INFO:tensorflow:loss = 43953.57, step = 6501 (27.408 sec)
INFO:tensorflow:global_step/sec: 3.61204
INFO:

In [ ]:
import matplotlib.pyplot as plt  
%matplotlib inline

# manually set the parameters of the figure to and appropriate size
plt.rcParams['figure.figsize'] = [14, 10]

loss_values = [ev['loss'] for ev in evaluations]  
training_steps = [ev['global_step'] for ev in evaluations]

plt.scatter(x=training_steps, y=loss_values)  
plt.xlabel('Training steps (Epochs = steps / 2)')  
plt.ylabel('Loss (SSE)')  
plt.show()  

In [ ]:
red = regressor.predict(input_fn=wx_input_fn(X_val,  
                                              num_epochs=1,
                                              shuffle=False))
predictions = np.array([p['predictions'][0] for p in pred])

print("The Explained Variance: %.2f" % explained_variance_score(  
                                            X_val, predictions))  
print("The Mean Absolute Error: %.2f degrees Celcius" % mean_absolute_error(  
                                            X_val, predictions))  
print("The Median Absolute Error: %.2f degrees Celcius" % median_absolute_error(  
                                            X_val, predictions))